This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = input()
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

## Generate Listings

In [4]:
llm = ChatOpenAI(
    model='gpt-3.5-turbo-0125', 
    temperature=0.5,
    max_tokens=2500)

prompt = '''
Generate 35 diverse and realistic real estate listings in the United States from different cities like Wahsington , NewYork, Florida etc.
Generated listings should be catchy and descriptive, so they encourage reviewers to buy such property.
Also they should contain facts, such as:
- price
- location
- available surface
- number of bedrooms and bathrooms
- property type
- amenities
- description of the neighborhood.

Separate listings by '---'.     
'''

In [5]:
result = llm.invoke(prompt)

print(result.content)

1. Luxurious penthouse in the heart of Manhattan, New York
Price: $5,000,000
Location: Upper East Side
Surface: 3,000 sq ft
Bedrooms: 4, Bathrooms: 4
Property type: Condo
Amenities: Rooftop terrace, concierge service, fitness center
Neighborhood: Upscale area with top restaurants and shopping

---
2. Waterfront estate in Miami Beach, Florida
Price: $8,500,000
Location: Star Island
Surface: 7,000 sq ft
Bedrooms: 6, Bathrooms: 7
Property type: Single-family home
Amenities: Private dock, pool, guest house
Neighborhood: Exclusive island with stunning views of the bay

---
3. Historic townhouse in Georgetown, Washington D.C.
Price: $2,500,000
Location: Georgetown
Surface: 2,500 sq ft
Bedrooms: 3, Bathrooms: 3
Property type: Townhouse
Amenities: Original hardwood floors, fireplace, garden
Neighborhood: Charming cobblestone streets with boutique shops and cafes

---
4. Modern condo with city views in San Francisco, California
Price: $1,200,000
Location: South of Market
Surface: 1,200 sq ft
Be

In [6]:
with open('real_est.txt', 'w') as listings_file:
    listings_file.write(result.content)

In [7]:
import chromadb
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="real_estate_listings")

loader = TextLoader("./real_est.txt")
files = loader.load()

text_splitter = CharacterTextSplitter(separator='\n---\n', chunk_size=400, chunk_overlap=0)
docs = text_splitter.split_documents(files)
len(docs)



31

In [8]:

embedding_model = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"], model="text-embedding-3-large")
db = Chroma.from_documents(docs, embedding_model, persist_directory='./chroma_db')


In [10]:
query = '''
I would like to live in one of state in the middle of US and I can spend up to $1.5 Million. 
I want to have an estate in gated community with a lot of amenities for children (e.g. school, playground).
Beside of that house should have at least 4 bedrooms, spacious kitchen and garden.
'''
docs = db.similarity_search(query)

for doc in docs:
    print(doc.page_content)

12. Ranch-style estate in Scottsdale, Arizona
Price: $2,000,000
Location: Paradise Valley
Surface: 4,000 sq ft
Bedrooms: 4, Bathrooms: 5
Property type: Single-family home
Amenities: Horse stables, outdoor kitchen, pool
Neighborhood: Upscale area with golf courses and desert landscapes
1. Luxurious penthouse in the heart of Manhattan, New York
Price: $5,000,000
Location: Upper East Side
Surface: 3,000 sq ft
Bedrooms: 4, Bathrooms: 4
Property type: Condo
Amenities: Rooftop terrace, concierge service, fitness center
Neighborhood: Upscale area with top restaurants and shopping
12. Equestrian Estate in Lexington, KY
Price: $3,000,000
Location: Lexington, KY
Surface: 10,000 sqft
Bedrooms: 7
Bathrooms: 5
Property type: Equestrian estate
Amenities: Horse stables, riding arena, guest house
Description: Live the equestrian lifestyle in this sprawling estate in Lexington with acres of land for horses and outdoor activities. Explore nearby horse farms and bourbon distilleries.
7. Urban Oasis in Br

## Creating Embedding & saving as Chroma

In [20]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # You can use "map_reduce" for more complex queries
    retriever=db.as_retriever()
)

## Input User Preferences

In [15]:
questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
input_dict={}
for i in questions:

    input_dict[i] = input(i)

How big do you want your house to be?What are 3 most important things for you in choosing this property?Good locality, Great view
Which amenities would you like?>3 Bedrooms
Which transportation options are important to you?Hospital near me
How urban do you want your neighborhood to be?Urban


In [18]:
input_context = ' '.join(f'Question - {key} /n Answer - {value}' for key, value in input_dict.items())


## Gnerate Output 

In [21]:
query = "Show me listings with 3 bedrooms under $500,000."
query = "Show me top 2 lowest priced listing for bedroom >4 "
response = qa_chain.run("Based on the below expectation of the buyer list down the top 5 real estate listings - Expectations -{input_context}")
print(response)

Based on the buyer's expectations, here are the top 5 real estate listings from the provided context:

1. Waterfront estate in Miami Beach, Florida
2. Luxurious penthouse in the heart of Manhattan, New York
3. Mountain retreat in Aspen, Colorado
4. Ski-in/ski-out chalet in Park City, Utah

Unfortunately, I cannot provide a fifth option as there are only four listings provided in the context.
